네이버 전처리
1. 중복제거(사업장명, 위도, 경도)
2. DB 구축을 위한 primary 키 삽입 string(사업장명, 위도, 경도)
3. primary key nan제거

In [24]:
import pandas as pd
import numpy as np
import re
import os

In [25]:
naver_df = pd.read_csv('../../naver_crw/DAOU/test_large_out_v3.csv', index_col=0, encoding='euc-kr')

In [26]:
print(naver_df.info())
naver_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6664 entries, 0 to 6663
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    6664 non-null   object 
 1   업소유형                    874 non-null    object 
 2   업태구분명                   6664 non-null   object 
 3   지번주소                    6653 non-null   object 
 4   도로명주소                   3986 non-null   object 
 5   가까운 지하철역                5774 non-null   object 
 6   지하철역과의 거리               6664 non-null   float64
 7   도보시간                    6664 non-null   float64
 8   위도                      5774 non-null   float64
 9   경도                      5774 non-null   float64
 10  별점                      310 non-null    float64
 11  방문자 리뷰수                 685 non-null    object 
 12  블로그 리뷰수                 513 non-null    object 
 13  한실수                     5434 non-null   float64
 14  양실수                     5347 non-null   

In [29]:
# .. 사업장명 중복 제거
# 중복 개수 확인
df_temp1 = naver_df.groupby(['사업장명', '위도', '경도'])['도로명주소'].count().reset_index(name='호실개수')
duplicate_name = df_temp1[df_temp1.호실개수 >= 2]['사업장명'] # 21개

naver_df_p1 = naver_df.drop_duplicates(['사업장명', '위도', '경도'])    # 6664 --> 6383

print(len(duplicate_name), naver_df_p1.shape[0], naver_df.shape[0])

21 6383 6664


In [30]:
# 프라이머리 키 생성
naver_df_p1['naver_pk'] = np.nan
for i in range(naver_df_p1.shape[0]):
    try:
        naver_df_p1['naver_pk'][i] = f"{naver_df_p1['사업장명'][i]}, {naver_df_p1['위도'][i]:.10f}, {naver_df_p1['경도'][i]:.10f}"
    except:
        naver_df_p1['naver_pk'][i] = np.nan


C:\Users\tjoeun\AppData\Local\Temp\ipykernel_13496\1562532280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naver_df_p1['naver_pk'] = np.nan
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_13496\1562532280.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naver_df_p1['naver_pk'][i] = f"{naver_df_p1['사업장명'][i]}, {naver_df_p1['위도'][i]:.10f}, {naver_df_p1['경도'][i]:.10f}"
C:\Users\tjoeun\AppData\Local\Temp\ipykernel_13496\1562532280.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

In [32]:
# 프라이머리 키 nan 제거
print(len(naver_df_p1[~pd.isna(naver_df_p1['naver_pk'])]))
naver_df_p1_remove_nan = naver_df_p1[~pd.isna(naver_df_p1['naver_pk'])]

6122


In [33]:
naver_df_p1_remove_nan.to_csv("../DAOU/test_large_out_v4", encoding='euc-kr')